# Plot Segmentation in Agriculture Using Computer Vision Techniques: A Scientific Approach

In the realm of precision agriculture, plot segmentation plays a pivotal role in crop management and yield optimization. This Jupyter Notebook presents a comprehensive workflow for segmenting agricultural plots by harnessing the power of computer vision and image processing techniques. We demonstrate the integration of image manipulation, filtering, and augmentation to extract meaningful insights from agricultural imagery.



## Table of Contents
1. [Setup and Data Preparation](#section1)
   - [Initial Imports](#subsection1-1)
   - [Drive Mounting](#subsection1-2)
2. [Execution of Processing Pipeline](#section2)
   - [Pipeline Configuration](#subsection2-1)
   - [Visualizing Objects](#subsection2-2)
   - [Loading Resources as `np.array`](#subsection2-3)
3. [Model Training](#section3)
4. [Model Evaluation and Visualization](#section4)
5. [Fine Tuning and Hyperparameter Optimization](#section5)
   - [Early Stopping](#subsection5-1)
   - [Optuna Optimization](#subsection5-2)

# Section 1: Setup and Data Preparation


## Initial Imports

Here we import necessary libraries such as OpenCV, NumPy, and Matplotlib. These libraries provide us with the tools required for image manipulation and visualization.


In [ ]:
!pip install optuna -q
!pip install imagehash -q
!pip install tensorflow -q
!pip install early_stopping -q

In [ ]:
import os
import sys
import math
import copy
import time
import torch
import random
import psutil
import optuna
import imagehash
import subprocess

import torch.nn.functional  as F
import keras                as K
import cv2                  as cv
import numpy                as np
import tensorflow           as tf
import torch.nn             as nn
import matplotlib.pyplot    as plt
import torch.optim          as optim

from google.colab           import drive
from torchvision            import models
from torchsummary           import summary
from scipy.ndimage          import convolve
from torch.utils.data       import DataLoader
from torchvision            import transforms
from keras.models           import Sequential
from PIL                    import Image, ImageTk
from tensorflow             import data as tf_data
from torch.utils.data       import DataLoader, Dataset
from keras                  import optimizers, callbacks, Model
from keras.layers           import Conv2D, Input, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation, SeparableConv2D, Conv2DTranspose, UpSampling2D, add


from Helper                 import *
from BaseImageProcess       import *
from NeuralNetwork          import *
from optuna.trial           import TrialState
from Visualizer             import Visualizer
from ImageDataManager       import ImageDataManager
from ProcessingPipeline     import ProcessingPipeline
from Loader                 import get_masks_and_images_as_np_array

## Drive Mounting

Here we mount the Google Drive to access the dataset.

In [ ]:
drive.mount('/content/drive')

# Section 2: Execution of Processing Pipeline

## Pipeline Configuration

Set up the processing pipeline by selecting the desired filters and augmentations. This configuration will determine how the images are processed and enhanced.


In [ ]:
pipeline = ProcessingPipeline()
pipeline.add_augmentations([Rotate(), Translate(), Flip(), BrightnessContrast(), RandomGaussianBlur(), MedianBlur()])

In [ ]:
base_masks_path = "/content/drive/MyDrive/Images/Masks"
base_inputs_path = "/content/drive/MyDrive/Images/Input"
image_data_manager = ImageDataManager(base_masks_path, base_inputs_path)

## Visualizing Objects

In [ ]:
Visualizer.visualize_objects(image_data_manager.objects)

## Loading resources

#### Parameters

In [ ]:
BATCH_SIZE = 1

CROP_SIZE = 120

RAW_IMAGE_SIZE = 1200

IMAGE_SIZE = (CROP_SIZE, CROP_SIZE)

TEST_SET_SIZE_AS_PERCENTAGE = 0.2

#### Resources

In [ ]:
masks_as_np_array = []
images_as_np_array = []

for key in image_data_manager.objects.keys():
    images = image_data_manager.objects[key]['images']
    mask = image_data_manager.objects[key]['mask']

    filtered_images = []

    for image in images:
        if image.shape[1] == RAW_IMAGE_SIZE:
            image = np.transpose(image, (1, 2, 0))
            if image.shape[2] == 1:
                image = np.repeat(image, 3, axis=2)
            filtered_images.append(image)

    image_data_manager.objects[key]['images'] = filtered_images

    image = filtered_images[-1]
    n_crop = image.shape[0] // CROP_SIZE
    images, masks, coordinates = pipeline.run(image, mask, crop_size=CROP_SIZE, n_crop=n_crop, n_augmented=0)

    masks_as_np_array.extend(masks)
    images_as_np_array.extend(images)

print(f"Loaded {len(images_as_np_array)} images")

# Section 3: Model Training

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = ResUNet(n_channels=3, n_classes=2, dropout=0.13677432839622955).to(device)

# Move model to GPU
model = model.to(device)

# Set the model in train mode
model.train()

# Define the preprocessing steps
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Split your data into training and testing sets
val_samples = int(len(images_as_np_array) * TEST_SET_SIZE_AS_PERCENTAGE)

train_x = images_as_np_array[:-val_samples]
train_y = masks_as_np_array[:-val_samples]

test_x = images_as_np_array[-val_samples:]
test_y = masks_as_np_array[-val_samples:]


# Create PyTorch Datasets
train_set = CustomDataset(
    images=train_x,
    masks=train_y,
    transform=preprocess
)

test_set = CustomDataset(
    images=test_x,
    masks=test_y,
    transform=preprocess
)

# Create PyTorch DataLoaders
train_loader = DataLoader(train_set, batch_size=16, shuffle=True)
test_loader = DataLoader(test_set, batch_size=16, shuffle=False)

# Define the optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.0013674142219634881)
criterion = nn.CrossEntropyLoss()

# Set the model in training mode
model.train()

NUM_EPOCHS = 2000

def dice_loss(pred, target, smooth = 1.):
    pred = pred.argmax(dim=1)
    intersection = (pred * target).sum(dim=(1,2))
    union = pred.sum(dim=(1,2)) + target.sum(dim=(1,2))

    dice = (2. * intersection + smooth) / (union + smooth)

    return 1 - dice.mean()


losses = []
dice_coeffs = []

best_coverage = 0.0
best_model_wts = copy.deepcopy(model.state_dict())

# Training loop
for epoch in range(NUM_EPOCHS):
    epoch_loss = 0
    epoch_dice = 0
    coverage = 0

    for i, (inputs, targets) in enumerate(train_loader):
        inputs = inputs.float().to(device)  # Convert inputs to Float format and move to GPU
        targets = targets.long().to(device)  # Ensure targets are Long format for CrossEntropyLoss and move to GPU

        # Forward pass
        # outputs = model(inputs)['out']
        outputs = model(inputs)

        # Calculate loss
        loss = criterion(outputs, targets)
        d_loss = dice_loss(torch.softmax(outputs, dim=1), targets)
        total_loss = loss + d_loss

        # Backward pass and optimization
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        # Calculate Dice coefficient
        _, preds = torch.max(outputs, 1)
        dice = dice_coefficient(preds, targets)
        epoch_dice += dice.item()
        epoch_loss += total_loss.item()

        current_coverage = calculate_coverage(preds, targets)
        if current_coverage > best_coverage:
            best_coverage = current_coverage
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(model, '/content/drive/My Drive/models/res_u_coverage'+str(current_coverage)+'_.pth')

    # Print loss and Dice coefficient every epoch
    epoch_loss = epoch_loss / len(train_loader)
    epoch_dice = epoch_dice / len(train_loader)
    losses.append(epoch_loss)
    dice_coeffs.append(epoch_dice)


    print(f"Epoch {epoch+1}/{NUM_EPOCHS}, Loss: {epoch_loss:.4f}, Dice Coefficient: {epoch_dice:.4f}, Coverage: {calculate_coverage(preds, targets)}")

## Finding Duplicates

In [ ]:
def find_duplicates(train_data, test_data):
    """Find duplicate images in the training and testing sets."""

    # Compute a hash for each image in the training set
    train_hashes = [imagehash.phash(Image.fromarray((img * 255).astype(np.uint8))) for img in train_data]

    # Compute a hash for each image in the testing set
    test_hashes = [imagehash.phash(Image.fromarray((img * 255).astype(np.uint8))) for img in test_data]

    # Find duplicates
    duplicates = [test for test in test_hashes if test in train_hashes]

    return duplicates

duplicates_x = find_duplicates(train_x, test_x)
duplicates_y = find_duplicates(train_y, test_y)

print(f"Duplicate images in train_x and test_x: {len(duplicates_x)}")
print(f"Duplicate masks in train_y and test_y: {len(duplicates_y)}")

In [ ]:
def visualize_data(images, masks, title):
    """Function to visualize images and masks"""
    n = 10  # number of samples to display
    fig, ax = plt.subplots(n, 2, figsize=(10, 20))

    for i in range(n):
        ax[i, 0].imshow(images[i])
        ax[i, 0].set_title(f'{title} Image {i+1}')
        ax[i, 1].imshow(masks[i], cmap='gray')
        ax[i, 1].set_title(f'{title} Mask {i+1}')

    plt.tight_layout()
    plt.show()

# Visualize training images and masks
visualize_data(train_x, train_y, 'Train')

# Visualize testing images and masks
visualize_data(test_x, test_y, 'Test')


# Section 4: Model Evaluation and Visualization

In [ ]:
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.plot(losses, label='Loss')
plt.title('Loss during training')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(dice_coeffs, label='Dice Coefficient')
plt.title('Dice Coefficient during training')
plt.xlabel('Epoch')
plt.ylabel('Dice Coefficient')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
coverage = calculate_coverage(preds, targets)
print(f"Coverage: {coverage:.2f}%")

# Manual Evaluating

In [ ]:
model = torch.load('/content/drive/My Drive/models/res_u_coverage97.26646307573702_.pth')
model.eval()


In [ ]:
def image_transform_predict(model, image_path, device, transform=transforms.Compose([
    transforms.Resize((120, 120)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])):
    # Load the image
    img = Image.open(image_path).convert("RGB")

    # Apply the transformations
    img_t = transform(img)

    # Create a mini-batch
    img_t = img_t.unsqueeze(0)

    # Move tensor to the device where your model is
    img_t = img_t.to(device)

    # Use your model to predict
    model.eval()
    with torch.no_grad():
        output = model(img_t)

    # Use argmax to get the most likely class for each pixel
    _, preds = torch.max(output, dim=1)

    # Move predictions to CPU and convert to numpy array
    preds = preds.cpu().numpy()

    # Plot the prediction
    plt.imshow(preds[0], cmap='gray')
    plt.show()


In [ ]:
image_transform_predict(model, "/content/drive/MyDrive/manual/1.png", device)
image_transform_predict(model, "/content/drive/MyDrive/manual/2.png", device)


# Section 5: Fine Tuning and Hyperparameter Optimization

## Early Stopping

In [ ]:
class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), 'checkpoint.pt')
        self.val_loss_min = val_loss

## Optuna Optimization

In [ ]:
def objective(trial):
    # Hyperparameters to tune
    dropout = trial.suggest_float("dropout", 0.1, 0.5)  # Dropout rate
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)  # Learning rate

    # Model, criterion, optimizer
    model = ResUNet(n_channels=3, n_classes=2, dropout=dropout).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    early_stopping = EarlyStopping(patience=10, verbose=True)  # Adjust patience as needed

    val_loss = np.inf  # Initialize val_loss

    for epoch in range(NUM_EPOCHS):
        epoch_loss = 0
        epoch_dice = 0

        model.train()
        for i, (inputs, targets) in enumerate(train_loader):
            inputs = inputs.float().to(device)
            targets = targets.long().to(device)

            outputs = model(inputs)

            loss = criterion(outputs, targets)
            d_loss = dice_loss(torch.softmax(outputs, dim=1), targets)
            total_loss = loss + d_loss

            optimizer.zero_grad()
            total_loss.backward()
            optimizer.step()

            _, preds = torch.max(outputs, 1)
            dice = dice_coefficient(preds, targets)
            epoch_dice += dice.item()
            epoch_loss += total_loss.item()

        epoch_loss = epoch_loss / len(train_loader)
        epoch_dice = epoch_dice / len(train_loader)

        # Validation
        model.eval()
        val_loss_temp = 0
        with torch.no_grad():
            for i, (inputs, targets) in enumerate(test_loader):
                inputs = inputs.float().to(device)
                targets = targets.long().to(device)

                outputs = model(inputs)

                loss = criterion(outputs, targets)
                d_loss = dice_loss(torch.softmax(outputs, dim=1), targets)
                total_loss = loss + d_loss

                val_loss_temp += total_loss.item()

        val_loss_temp = val_loss_temp / len(test_loader)

        # Early stopping
        early_stopping(val_loss_temp, model)
        if early_stopping.early_stop:
            print("Early stopping")
            break

        val_loss = val_loss_temp  # Update val_loss only if model was evaluated

    return val_loss  # Now val_loss is never None



study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100, timeout=600)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))


In [ ]:
torch.save(model, '/content/drive/My Drive/model_res_u_coverage_96_38.pth')